In [1]:
import logging
import re
import pandas as pd
import numpy as np

from dataclasses import dataclass
from datetime import date, datetime, timezone, timedelta
from typing import Dict, List
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import ASYNCHRONOUS
from pandas import DataFrame
from IPython.display import clear_output
import matplotlib.pyplot as plt
from termcolor import colored

#from scc_co2_buz.domain.models.well_data import IO
from scc_co2_buz.helpers.utils.io_normalize import db_types 
from scc_co2_buz.helpers.utils.get_utc import get_utc_now
from scc_co2_buz.domain.models.tag import TagModel

In [2]:
# due to changes during tests some well tags are changed
map_to_tag = {'9-BUZ-11-RJS': 'well injection C', 
              '9-BUZ-35-RJS':'well injection D',
              '9-BUZ-3-RJS':'well injection C',
              '9-BUZ-4-RJS':'well injection D'}

class InfluxDbConfig:
    """"
    ConfigClass for InfluxDb
    """
    def __init__(self) -> None:
        self.url = "http://10.136.226.25:8086"
        self.token = "ganimede@2050"
        self.org = "PETROBRAS"
        self.bucket = "vip"
        self.write_options = ASYNCHRONOUS

class InfluxDBadapter:
    def __init__(self, config: InfluxDbConfig) -> None:
        self.config = config
        self.client = InfluxDBClient(config.url, config.token, debug=None, org=config.org)
        self.log = logging.getLogger(name='repository')
        print("connecting to P-77 database...")
    def closeConnection(self):
        self.client.close()
        self.log.info('connection to database closed')
    def AddTag(self, tag: TagModel) -> Dict[bool, Exception]:
        try:
            _point = self.makeTagPoint(tag)
            self.client.write_api(
                write_options=self.config.write_options).write(bucket=self.config.bucket, record=[_point], org=self.config.org)
            self.log.info('tag ' + tag.tag + ' sent to database')

        except Exception as e:
            msg = 'error sending tag ' + tag.tag + ' to database ' + e.args
            self.log.error(msg)
            return None, e
        return (True, None)

    def addAllTags(self, tags: List[TagModel]) -> Dict[bool, Exception]:
        try:
            points = [self.makeTagPoint(t) for t in tags]
            db = self.client.write_api(
                write_options=self.config.write_options).write(bucket=self.config.bucket, org=self.config.org, record=points)
            db.get()
        except Exception as e:
            msg = 'error sending tags to database ' + str(e)
            self.log.error(msg)
            return None, e
        return (True, None)

    def makeTagPoint(self, tag: List[TagModel]) -> Point:
        tag_field = 'DIAGNOSTIC' if tag.io in [
            IO.HEARTBEAT, IO.GENERALDIAG] else 'well'
        return Point("IO").tag(tag_field, tag.wn).field(tag.io, db_types[tag.io](tag.value)).time(tag.timestamp)

    def getData(self, start: date, end: date, tags:[TagModel] = None) -> DataFrame:
        params = {
            "start":  datetime.fromtimestamp(datetime.timestamp(start), tz=timezone.utc),
            "stop": datetime.fromtimestamp(datetime.timestamp(end), tz=timezone.utc),
            "bucket": self.config.bucket
        }
        filter_io = ''
        db_cols = list(set([tag.io for tag in tags]))
        for io in db_cols:
            if io != db_cols[-1]:
                filter_io = filter_io + f'r[\"_field\"] == \"{io}\" or '
            else:
                filter_io = filter_io + f'r[\"_field\"] == \"{io}\"'
        query = 'from(bucket: bucket)\
            |> range(start: start, stop: stop)\
            |> filter(fn: (r) => r["_measurement"] == "IO")\
            |> filter(fn: (r) => ' + filter_io + ')\
            |> yield(name: "last")'
        query_api = self.client.query_api()
        result = query_api.query(
            org=self.config.org, query=query, params=params)
        results = []
        for table in result:
            for record in table.records:
                results.append((datetime.fromtimestamp(
                    record.get_time().timestamp(), tz=timezone.utc), record.get_value(), record['well'], record.get_field()))
        df = pd.DataFrame(data=results, columns=['time', 'value', 'oil_well', 'io'])
        df2 = df.pivot(index=['time', 'oil_well'], columns=['io'], values=['value']).reset_index()
        df2.columns = [c[{'time': 0, 'oil_well': 0, 'value': 1}[c[0]]] for c in df2.columns]
        return df2
    
@dataclass
class Color:
   PURPLE = '\033[1;35;48m'
   CYAN = '\033[1;36;48m'
   BOLD = '\033[1;37;48m'
   BLUE = '\033[1;34;48m'
   GREEN = "\033[1;32;48m"
   YELLOW = '\033[1;33;48m'
   RED = '\033[1;31;48m'
   BLACK = '\033[1;30;48m'
   UNDERLINE = '\033[4;37;48m'
   END = '\033[1;37;0m'

def print_colored(msg:str, color: Color = Color.GREEN):
    print("{} {} {}".format(color,msg, Color.END))
def calcNormStd(df, well, pit, delta = timedelta(seconds=60)):
    start = df['time'].min()
    end = start + delta
    max_norm_std = -1.0
    
    time_limit = df.iloc[-1]['time'] - delta
    iteration = 0
    total_iterations = int((time_limit.timestamp() - start.timestamp()) / 60 )    
    values = []
    iterations = []
    #time_limit = 1000 * delta + start
    max_sample_data = None
    
    while start <= time_limit:
        mask = (df['oil_well'] == well) & (df['time'] >= start) & (df['time'] < end) 
        current_data = df.loc[mask][pit]
        current_norm_std = current_data.std() / current_data.mean()
        iteration += 1
        clear_output(wait=True)
        msg = f'iteration = {iteration} \n > progress = {100*iteration/total_iterations if iteration < total_iterations else 100.00} %'
        color = "green"
        print_colored(msg, Color.GREEN)
        if current_norm_std > max_norm_std:
            max_norm_std = current_norm_std 
            values.append(max_norm_std) 
            iterations.append(iteration)
            print(max_norm_std)
            max_sample_data = current_data
        start = start + delta
        end = end + delta
        
    df_std = pd.DataFrame({'iteration': iterations, 'variation': values})
    df_std.plot(x='iteration', y='variation', kind='line')
    plt.show()    
    return f'coefficient of variation = {100*max_norm_std} %'

def toFloat(x):
    
    if type(x) == float or type(x) == int:
        return x
    
    x = x.strip()
    x = x.replace(' ','')
    
    result1 = None
    result2 = None
    
    try:
        result1 = re.findall(r'\d+\.\d+', x)
        result2 = re.findall(r'\d+', x)
    except Exception as e:
        return np.nan
        
    if len(result1) > 0 and type(result1) == list:
        return float(result1[0])
    elif len(result2) > 0 and type(result2) == list:
        return float(result2[0])
    else:
        # comm fail from PI Web API
        return np.nan

def checkSupress(supression, time_):
    variation = 100 * ( supression.max() - supression.min() ) / (supression.mean() + 0.0000000001)
    if variation >= 2.0:
        print_colored(' > supressed by ' + supression.name + '\n  in time: ' + str(time_) + '\n  with variation : ' + str(variation) + ' %', Color.YELLOW)
        return True
    return False   

def normalize_data(df):
    for col in df.columns:
        if col not in ['time','oil_well']:
            if 'SDY' in col:
                df[col] = df[col].apply(lambda x: 1 if x  else 0)
            else:
                df[col] = df[col].apply(lambda x: toFloat(x))
                #std = df[col].std()
                #mean = df[col].mean()
                #df[col] = df[col].apply(lambda x: (x - mean) / (std + 0.000000000001))
        else:
            if col == 'time':
                #df[col] = df[col].apply(lambda x: datetime.strptime(x.replace('nov','11'), '%d-%m-%y %H:%M:%S'))
                df[col] = df[col].apply(lambda x: datetime.strptime(x.split('+')[0], '%Y-%m-%d %H:%M:%S.%f') - timedelta(hours=3)) 

            ...
    #df = df.dropna()
    return df

class Signal:
    def __init__(self):
        self.io_input = []
        self.io_ouput = []
        self.alarm_values = [0,0,0,0,0,0]
        self.sample_values = [0,0,0,0,0,0]
        self.values = [0,0,0,0,0,0,0,0,0]
        
    def check_alarm(self, df, instrument_supress, time_, i, v, set_point_ll = -67 * 100):
        
        self.alarm_values.append(1.0 if v <= set_point_ll else 0.0 ) 
        self.alarm_values.pop(0)
        self.sample_values.append(v)
        self.sample_values.pop(0)  
        
        check = 0         

        if sum(self.alarm_values) == 6:   
    
            # select data for supression
            start = time_ - timedelta(seconds = 120) 
            end = time_
            
            ## code repetition for n supressions
            mask = (df[instrument_supress].notnull()) & (df['time'] >= start) & (df['time'] <= end)
            supression = df.loc[mask][instrument_supress]
            
            print(len(supression))
            print_colored(f'{start} , {end}',Color.BLUE)
            
            # change check to 1 if supression is enable
            if not checkSupress(supression, time_):
                check = 1
                
            ## code repetition for n supressions
            
            # print alarm
            color = Color.RED if check == 1 else Color.GREEN
            print_colored('iter = '+ str(i), color)
            print_colored('time = '+ str(time_), color)
            print_colored('signal inputs '+ str(self.values) + ' kPa', color)
            print_colored('signal outputs'+ str(self.sample_values) + ' kPa/min',color)
            self.alarm_values = [0,0,0,0,0,0]
            self.sample_values = [0,0,0,0,0,0]
       
        
        return check
    
    def signalFilter(self,df,instrument,instrument_supress,coef = [-4,-3,-2,-1,0,1,2,3,4],gain = 1.0):
        
        delta_sec = 5       
        mask = df[instrument].notnull()
        current_data = df.loc[mask][instrument]
        current_time = df.loc[mask]['time']
        
        total_iterations = int(len(current_data) * 1)
        iter_out = []
        status_in_alarm = 0
        alarms = []
        
        print_colored('total iterations ' + str(total_iterations), Color.BLUE )
        
        for i in range(total_iterations):
            iter_out.append(i)
            self.io_input.append(current_data.iloc[i]) 
            out_ = 0
            self.values.append(current_data.iloc[i])
            self.values.pop(0)
            for c, v in  zip(coef,self.values):
                out_  +=  c * v
            filtered = gain * out_ / delta_sec
            self.io_ouput.append(filtered) 
            alarms.append(self.check_alarm(df,instrument_supress, current_time.iloc[i], i, filtered))
        
        # plot fig size
        size = (18,6)
        # alarm
        alm = DataFrame({'time':iter_out,'alarms': alarms})
        alm.plot(x='time',y='alarms', kind='line', figsize=size)
        
        # input signal
        df_input = DataFrame({'time': iter_out, instrument: self.io_input})
        df_input.plot(x='time', y=instrument, kind='line', figsize=size)
        
        # supress signal
        mask = df[instrument_supress].notnull()
        df.loc[mask].plot(x='time', y=instrument_supress, kind='line', figsize=size)        
        
        # filtered signal
        result = DataFrame({'time':iter_out,'filtered_signal': self.io_ouput})
        result.plot(x='time',y='filtered_signal', kind='line', figsize=size)        
        
        # print alarms
        total_alarms = alm.sum().values[1]
        color = Color.RED if total_alarms >= 1 else Color.GREEN
        print_colored('total alarms =  ' + str(total_alarms), color)
        
        return result 
    
def make_well_data(df, wells=['well injection C',  'well injection D']):
    df_result = DataFrame()
    for well in wells:
        l = well.split(' ')[2]
        mask = (df['oil_well'] == well)
        df_result = df_result.append(DataFrame({
        'time' : df.loc[mask]['time'].values,
        'ZIT_1244002' + l: df.loc[mask]['ZIT_1244002'].values,
        'PIT_1244007' + l: df.loc[mask]['PIT_1244007'].values,
        'PIT_1244008' + l: df.loc[mask]['PIT_1244008'].values,
        'SDY_1210002' + l: df.loc[mask]['SDY_1210002'].values}))
    return df_result



In [3]:
# ESTADO-POCO, SDV, ZSH = ZSH_1210002C/D
# ABER-CKP, CHOKE = ZIT_1244002C/D
# CMD = SDY_1210002C/D
# ZSL = ZSL_1210002C/D
columns = {'P-JUZ-CKP-A': "PIT-1244007", 
           'P-JUZ-CKP-B': "PIT-1244008",
           'ESTADO-POCO':'ZSH_1210002_1',
           'SDV':'ZSH_1210002_2',
           'ZSH': 'ZSH_1210002_3',
           'ZSL': 'ZSL_1210002',
           'ABER-CKP': 'ZIT_1244002_1',
           'CHOKE': 'ZIT_1244002_2',
           'CMD': 'SDY_1210002'}

repo = InfluxDBadapter(config=InfluxDbConfig())
tags = list( map( lambda io: TagModel(paths=None,tag=None, io=io), columns.keys()))

connecting to P-77 database...


In [4]:
end = get_utc_now()
start = get_utc_now() - timedelta(seconds=60*60*24*15)#timedelta(seconds=259200)
# comentando para não ficar baixando dados do banco
df = repo.getData(start=start,end=end,tags=tags)
df['oil_well'] = df['oil_well'].apply(lambda x: map_to_tag[x])
df = df.rename(columns=columns)
df_select = df.query('oil_well=="well injection C" or oil_well == "well injection D"')
df_select

KeyError: '7-BUZ-30-RJS'

In [7]:
df2 = df
df['oil_well'] = df['oil_well'].apply(lambda x: map_to_tag[x])
df = df.rename(columns=columns)
df_select = df.query('oil_well=="well injection C" or oil_well == "well injection D"')
df_select

KeyError: '7-BUZ-30-RJS'

In [15]:
mask = (df2['oil_well'].apply(lambda v: v in map_to_tag.keys()))
df = df2.loc[mask].copy()
df['oil_well'] = df['oil_well'].apply(lambda x: map_to_tag[x])
df = df.rename(columns=columns)
df_select = df.query('oil_well=="well injection C" or oil_well == "well injection D"')
df_select

,time,oil_well,ZIT_1244002_1,ZIT_1244002_2,SDY_1210002,ZSH_1210002_1,PIT-1244007,PIT-1244008,ZSH_1210002_2,ZSH_1210002_3,ZSL_1210002
4,2022-11-16 20:23:26.263000+00:00,well injection D,40.381248,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
9,2022-11-16 20:24:28.648958+00:00,well injection D,40.381248,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
14,2022-11-16 20:25:23.823225+00:00,well injection D,40.381248,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
19,2022-11-16 20:26:24.164761+00:00,well injection D,40.381248,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
24,2022-11-16 20:27:26.006816+00:00,well injection D,40.381248,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
947187,2022-12-01 20:22:35.902700+00:00,well injection C,NaN,0.0,False,NaN,31258.5,31365.839844,NaN,False,False
947188,2022-12-01 20:22:35.902700+00:00,well injection D,NaN,NaN,False,NaN,31297.140625,31249.910156,NaN,False,NaN
947189,2022-12-01 20:22:44.611352+00:00,well injection D,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947190,2022-12-01 20:22:45.373740+00:00,well injection C,NaN,0.0,False,NaN,31249.910156,31365.839844,NaN,False,False


In [16]:
df_select.to_csv("output_data.csv")
#calcNormStd(df, 'well injection D', 'PIT_1244008')

In [17]:
df = pd.read_csv("output_data.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (5,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df = pd.read_csv("output_data.csv")
df = normalize_data(df)
df = make_well_data(df)
df.head()

In [ ]:
result = Signal().signalFilter(df,'PIT_1244007D' , 'ZIT_1244002D')

In [ ]:
s = Signal()
result = s.signalFilter(df,'PIT_1244007C' , 'SDY_1210002C')